## Điều kiện tiên quyết

- GPU NVIDIA (>=12GB được khuyến nghị) cùng bộ driver CUDA hiện hành và bản `torch` tương thích
- Python 3.10/3.11 cùng `ffmpeg` trong `PATH` để xử lý âm thanh
- Bộ dữ liệu cục bộ chứa các đoạn `.wav` (16 kHz-24 kHz) với một bản chép cho mỗi câu
- Đủ dung lượng đĩa để lưu checkpoints/logs trong `outputs/`
- (Tùy chọn) Tài khoản Weights & Biases nếu bạn đổi `dashboard_logger`

In [ ]:
from pathlib import Path
import os

PROJECT_ROOT = Path(os.getenv("XTTS_PROJECT_ROOT", Path.cwd())).resolve()
DATASET_ROOT = Path(os.getenv("XTTS_DATASET_ROOT", PROJECT_ROOT / "data" / "vi_dataset")).resolve()
TRAIN_METADATA = Path(os.getenv("XTTS_TRAIN_META", DATASET_ROOT / "metadata_train.csv"))
VAL_METADATA = Path(os.getenv("XTTS_VAL_META", DATASET_ROOT / "metadata_val.csv"))
OUTPUT_DIR = Path(os.getenv("XTTS_OUTPUT_DIR", PROJECT_ROOT / "outputs" / "xtts_vi_ft")).resolve()
CONFIG_TEMPLATE = Path(os.getenv("XTTS_CONFIG_TEMPLATE", PROJECT_ROOT / "models" / "config.json")).resolve()
CUSTOM_CONFIG = OUTPUT_DIR / "config.xtts.vi.json"

LANGUAGE = os.getenv("XTTS_LANGUAGE", "vi")
RUN_NAME = os.getenv("XTTS_RUN_NAME", "xtts_vi_finetune")

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Thư mục dự án  : {PROJECT_ROOT}")
print(f"Thư mục dữ liệu: {DATASET_ROOT}")
print(f"Manifest train : {TRAIN_METADATA}")
print(f"Manifest val   : {VAL_METADATA}")
print(f"Thư mục output : {OUTPUT_DIR}")
print(f"Tệp config gốc : {CONFIG_TEMPLATE}")
print(f"Config tùy chỉnh: {CUSTOM_CONFIG}")
print(f"Mã ngôn ngữ    : {LANGUAGE}")
print(f"Tên lượt chạy  : {RUN_NAME}")

if not CONFIG_TEMPLATE.exists():
    raise FileNotFoundError(f"Thiếu tệp config gốc: {CONFIG_TEMPLATE}")
if not DATASET_ROOT.exists():
    print("CẢNH BÁO: DATASET_ROOT chưa tồn tại. Hãy cập nhật XTTS_DATASET_ROOT trước khi huấn luyện.")
if not TRAIN_METADATA.exists():
    print("THÔNG BÁO: Không tìm thấy TRAIN_METADATA. Hãy tạo manifest bằng cell tương ứng trước khi huấn luyện.")
if not VAL_METADATA.exists():
    print("THÔNG BÁO: Không tìm thấy VAL_METADATA. Trình huấn luyện sẽ tự chia tập eval.")

In [ ]:
# Cài đặt phụ thuộc của repo và runtime (chạy một lần cho mỗi môi trường).
%pip install --upgrade pip wheel setuptools
%pip install -e .
%pip install tensorboard pandas soundfile mutagen

## Định dạng manifest dữ liệu

Các bộ nạp XTTS cần manifest phân tách bằng ký tự `|` để ánh xạ câu thoại và bản chép. Dùng formatter `coqui` dựng sẵn là cách nhanh nhất:
```
audio_file|text|speaker_name
wavs/utt_00001.wav|Xin chào tất cả mọi người.|speaker_0001
wavs/utt_00002.wav|Chúc bạn một ngày tốt lành.|speaker_0001
```
- đặt manifest trong `DATASET_ROOT`
- đặt `language` về đúng mã ISO (mặc định ở đây là `vi`)
- thư mục theo từng người nói hoặc cột `speaker_name` rõ ràng giúp cân bằng lấy mẫu
- bảo đảm tập kiểm định không trùng với manifest huấn luyện

In [ ]:
import csv
from typing import Tuple

def build_manifest_from_transcripts(
    dataset_root: Path,
    manifest_path: Path,
    transcript_suffix: str = ".txt",
    speaker_from_parent: bool = True,
 ) -> Tuple[Path, int]:
    """Quét các tệp wav và bản chép cùng tên để tạo manifest tương thích `coqui`.

    Mỗi tệp wav cần một tệp văn bản có cùng tên gốc.
    """
    dataset_root = Path(dataset_root)
    manifest_path = Path(manifest_path)
    rows = []
    for wav_path in sorted(dataset_root.rglob("*.wav")):
        txt_path = wav_path.with_suffix(transcript_suffix)
        if not txt_path.exists():
            continue
        text = txt_path.read_text(encoding="utf-8").strip()
        if not text:
            continue
        speaker_name = wav_path.parent.name if speaker_from_parent else dataset_root.name
        rel_wav = wav_path.relative_to(dataset_root)
        rows.append([rel_wav.as_posix(), text, speaker_name])
    manifest_path.parent.mkdir(parents=True, exist_ok=True)
    with manifest_path.open("w", encoding="utf-8", newline="") as f:
        writer = csv.writer(f, delimiter="|")
        writer.writerow(["audio_file", "text", "speaker_name"])
        writer.writerows(rows)
    print(f"Đã ghi {len(rows)} dòng vào {manifest_path}")
    return manifest_path, len(rows)

# Ví dụ sử dụng (bỏ comment sau khi đã có dữ liệu):
# build_manifest_from_transcripts(DATASET_ROOT, TRAIN_METADATA)

In [ ]:
import pandas as pd
import soundfile as sf
from IPython.display import display

def describe_manifest(manifest_path: Path, dataset_root: Path, sample_size: int = 5):
    manifest_path = Path(manifest_path)
    dataset_root = Path(dataset_root)
    if not manifest_path.exists():
        raise FileNotFoundError(f"Không tìm thấy manifest: {manifest_path}")
    df = pd.read_csv(manifest_path, sep="|")
    display(df.head(min(sample_size, len(df))))
    durations = []
    max_rows = min(len(df), max(50, sample_size))
    for wav_rel in df["audio_file"].head(max_rows):
        wav_path = dataset_root / wav_rel
        if not wav_path.exists():
            continue
        with sf.SoundFile(wav_path) as snd:
            durations.append(len(snd) / snd.samplerate)
    hours = sum(durations) / 3600 if durations else 0.0
    speaker_count = df["speaker_name"].nunique() if "speaker_name" in df.columns else "n/a"
    print(f"Số dòng        : {len(df)}")
    print(f"Số speaker duy nhất: {speaker_count}")
    print(f"Giờ audio xem trước: {hours:.2f} (trên {len(durations)} tệp)")
    return df

if TRAIN_METADATA.exists():
    train_df = describe_manifest(TRAIN_METADATA, DATASET_ROOT)
else:
    print("Thiếu manifest train. Hãy tạo bằng cell xây manifest.")

if VAL_METADATA.exists():
    val_df = describe_manifest(VAL_METADATA, DATASET_ROOT)
else:
    print("Sử dụng chia eval tự động (không có VAL_METADATA).")

In [ ]:
import json
import shutil
from copy import deepcopy
from datetime import datetime

def stage_manifest(manifest_path: Path) -> str:
    manifest_path = Path(manifest_path)
    if not manifest_path.exists():
        raise FileNotFoundError(f"Không tìm thấy manifest: {manifest_path}")
    target = DATASET_ROOT / manifest_path.name
    if manifest_path.resolve() != target.resolve():
        target.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy2(manifest_path, target)
        print(f"Đã sao chép {manifest_path} -> {target}")
    return target.name

train_meta_name = stage_manifest(TRAIN_METADATA)
val_meta_name = stage_manifest(VAL_METADATA) if VAL_METADATA.exists() else ""

with CONFIG_TEMPLATE.open("r", encoding="utf-8") as f:
    template = json.load(f)

config = deepcopy(template)
if not config.get("datasets"):
    config["datasets"] = [{}]

config["run_name"] = RUN_NAME
config["run_description"] = f"{RUN_NAME} ({datetime.utcnow().isoformat()}Z)"
config["output_path"] = str(OUTPUT_DIR)
config["dashboard_logger"] = os.getenv("XTTS_LOGGER", config.get("dashboard_logger", "tensorboard"))
config["epochs"] = int(os.getenv("XTTS_EPOCHS", config.get("epochs", 5)))
config["batch_size"] = int(os.getenv("XTTS_BATCH_SIZE", config.get("batch_size", 2)))
config["eval_batch_size"] = int(os.getenv("XTTS_EVAL_BATCH_SIZE", config.get("eval_batch_size", config.get("batch_size", 2))))
config["lr"] = float(os.getenv("XTTS_LR", config.get("lr", 5e-6)))
config["mixed_precision"] = os.getenv("XTTS_MIXED_PRECISION", str(config.get("mixed_precision", False))).lower() == "true"

dataset_cfg = config["datasets"][0]
dataset_cfg["formatter"] = dataset_cfg.get("formatter") or "coqui"
dataset_cfg["dataset_name"] = RUN_NAME
dataset_cfg["path"] = str(DATASET_ROOT)
dataset_cfg["meta_file_train"] = train_meta_name
dataset_cfg["meta_file_val"] = val_meta_name
dataset_cfg["language"] = LANGUAGE
dataset_cfg["phonemizer"] = dataset_cfg.get("phonemizer", "")
dataset_cfg["ignored_speakers"] = None
dataset_cfg["meta_file_attn_mask"] = dataset_cfg.get("meta_file_attn_mask", "")
config["datasets"][0] = dataset_cfg

with CUSTOM_CONFIG.open("w", encoding="utf-8") as f:
    json.dump(config, f, indent=2)

print(f"Đã ghi config tùy chỉnh vào {CUSTOM_CONFIG}")
print(f"Formatter dữ liệu : {dataset_cfg['formatter']}")
print(f"Đường dẫn train   : {dataset_cfg['path']}/{dataset_cfg['meta_file_train']}")
print(f"Đường dẫn eval    : {dataset_cfg['path']}/{dataset_cfg['meta_file_val'] or 'tự chia'}")

In [ ]:
from TTS.config import load_config
from TTS.tts.datasets import load_tts_samples

cfg = load_config(str(CUSTOM_CONFIG))
train_samples, eval_samples = load_tts_samples(
    cfg.datasets,
    eval_split=cfg.run_eval,
    eval_split_max_size=cfg.eval_split_max_size,
    eval_split_size=cfg.eval_split_size,
    )

print(f"Số mẫu train : {len(train_samples)}")
print(f"Số mẫu eval  : {len(eval_samples) if eval_samples is not None else 0}")
if train_samples:
    preview = train_samples[0]
    print("Các khóa ví dụ:", list(preview.keys()))
    print("Trích đoạn văn bản:", preview['text'][:120])

In [ ]:
import shlex
import subprocess
import sys

train_script = PROJECT_ROOT / "TTS" / "bin" / "train_tts.py"
continue_path = os.getenv("XTTS_CONTINUE_PATH")
cmd = [
    sys.executable,
    str(train_script),
    "--config_path",
    str(CUSTOM_CONFIG),
]
if continue_path:
    cmd += ["--continue_path", continue_path]

print("Lệnh huấn luyện:")
print(" ".join(shlex.quote(str(token)) for token in cmd))

LAUNCH_TRAINING = False  # Đặt True để chạy trực tiếp trong notebook.
if LAUNCH_TRAINING:
    process = subprocess.run(cmd, cwd=PROJECT_ROOT, check=False)
    print(f"Tiến trình huấn luyện kết thúc với mã {process.returncode}")
else:
    print("Chế độ mô phỏng (dry-run). Chuyển LAUNCH_TRAINING sang True khi bạn sẵn sàng.")

## Theo dõi & tiếp tục

- Đặt `XTTS_CONTINUE_PATH` tới thư mục checkpoint (`output/<run_name>`) để khôi phục phiên huấn luyện bị gián đoạn.
- Dùng TensorBoard để quan sát loss, gradient, attention và bản xem trước audio.
- Nếu bật logging Weights & Biases, đặt `dashboard_logger` = `wandb` và khai báo `WANDB_API_KEY`.

In [ ]:
# Mở TensorBoard ngay trong notebook (dừng cell để giải phóng cổng).
logdir = OUTPUT_DIR.as_posix()
%load_ext tensorboard
%tensorboard --logdir {logdir}

## Suy luận hậu huấn luyện

Chỉ định checkpoint đã huấn luyện (thường là `best_model.pth` hoặc `checkpoint_<step>.pth`) và cung cấp tệp tham chiếu giọng nói để clone. File demo sẽ được lưu cạnh thư mục kết quả của lượt chạy.

In [ ]:
import torch
from pathlib import Path
from TTS.api import TTS as CoquiTTS

checkpoint_override = os.getenv("XTTS_CHECKPOINT")
if checkpoint_override:
    checkpoint_path = Path(checkpoint_override)
else:
    candidates = sorted(OUTPUT_DIR.glob("best_model*.pth")) or sorted(OUTPUT_DIR.glob("checkpoint_*model.pth"))
    if not candidates:
        raise FileNotFoundError("Không tìm thấy checkpoint trong OUTPUT_DIR.")
    checkpoint_path = candidates[-1]

speaker_ref = Path(os.getenv("XTTS_SPEAKER_REF", ""))
if speaker_ref and speaker_ref.exists():
    speaker_wav = str(speaker_ref)
else:
    speaker_wav = None

sample_text = os.getenv("XTTS_TEST_SENTENCE", "Xin chào, đây là bản kiểm thử XTTS sau khi fine-tune.")
output_wav = OUTPUT_DIR / "sample_inference.wav"

tts_model = CoquiTTS(
    model_path=str(checkpoint_path),
    config_path=str(CUSTOM_CONFIG),
    progress_bar=False,
    gpu=torch.cuda.is_available(),
)

tts_model.tts_to_file(
    text=sample_text,
    speaker_wav=speaker_wav,
    language=LANGUAGE,
    file_path=str(output_wav),
)
print(f"Đã lưu audio demo tại {output_wav}")